<a href="https://colab.research.google.com/github/krishnamani77/TF2_Notebooks/blob/master/Sentiment-analysis-IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import keras.datasets

Using TensorFlow backend.


In [0]:
# IMDB Sentiment analysis
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

In [0]:
word_index = keras.datasets.imdb.get_word_index()
reverse_index = {}
for k, v in word_index.items():
  reverse_index[v+3] = k

In [94]:
print(len(word_index), len(reverse_index))

88584 88584


In [95]:
print(X_train[0])
print(y_train[:10])
print(y_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
[1 0 0 1 0 0 1 0 1 0]
1


In [96]:
print(X_train[0].__class__)
print(y_train[0].__class__)

<class 'list'>
<class 'numpy.int64'>


In [97]:
largest = 0
for line in X_train:
  if largest < len(line):
    largest = len(line)
  i += 1 
print(largest)

2494


In [0]:
for id, token in enumerate(("<pad>", "<sos>", "<unk")):
  reverse_index[id] = token

In [99]:
s = ' '.join([reverse_index[idx] for idx in X_train[0][:10]])

print(s)

<sos> this film was just brilliant casting location scenery story


In [0]:
vocab_size = len(word_index)

In [0]:
import numpy as np
def vectorize(sequences, dimension = 10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results

X_train = vectorize(X_train, 10000)
y_train = np.array(y_train).astype("float32")
X_test = vectorize(X_test, 10000)
y_test = np.array(y_test).astype("float32")

In [102]:
print(X_train[0])
print(y_train[:10])

[0. 1. 1. ... 0. 0. 0.]
[1. 0. 0. 1. 0. 0. 1. 0. 1. 0.]


In [0]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

embed_size = 128
embedding = Embedding(vocab_size, embed_size, input_shape=[None])
hidden1 = GRU(128, return_sequences = True)
hidden2 = GRU(128)
dense = Dense(1, activation='sigmoid')
model = Sequential([embedding, hidden1, hidden2, dense])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs=5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
